In [12]:
import datetime
import time

In [97]:
def new_quarter(quarter: int, year: int, delta_time: str) -> (int, int):
    if delta_time not in ['q', 'y', '3y', '5y']:
        return (10, 0)
    else:
        if delta_time == 'y':
            return (quarter, year-1)
        elif delta_time == '3y':
            return (quarter, year-3)
        elif delta_time == '5y':
            return (quarter, year-5)
        else:
            if quarter == 0:
                return (3, year - 1)
            else:
                return (quarter - 1, year)

In [70]:
def which_quarter(date: datetime) -> str:
    month = date.month
    if month <= 3:
        return 0
    elif month <= 6:
        return 1
    elif month <= 9:
        return 2
    else:
        return 3

In [71]:
def creating_adsh_dict(file_to_open: str) -> (list, dict):
    
    adsh_dict = {}
    
    with open(file_to_open,'rb') as file:
        counter = 0
        error1 = 0
        for a_line in file:
            
            try:
                a_line = str(a_line, 'utf-8')
            except:
                error1 += 1
            
            line_list = a_line.strip().split('\t')
            adsh = line_list[0] 
            cik = line_list[1]
            
            try:
                name = line_list[2].replace(",", "")
            except:
                name = line_list[2]
            
            filed = line_list[29]
            
            if counter > 0:    
                year = int(filed[:4])
                month = int(filed[4:6])
                day = int(filed[6:])
                filed = datetime.date(year, month, day)
                
            
            if counter < 1:
                headers = [cik, name, filed]
                
            else:
                adsh_dict[adsh] = [cik, name, filed]
                
            counter += 1
    return headers, adsh_dict

In [137]:
def cleaning_num_file(file_to_open: str, categories: list, num_files: int, company_dict: dict) -> dict:
    
    headers, adsh_dict = creating_adsh_dict('sub'+ file_to_open[3:])
    
    with open(file_to_open, 'rb') as file:
        counter = 0
        error1 = 0
        for a_line in file:
            
            try:
                a_line = str(a_line, 'utf-8') 
                line_list = a_line.split("\t")

                if line_list[1] in sub_categories and len(line_list[3]) == 0:
                    adsh = line_list[0]
                    tag = line_list[1]

                    ddate = line_list[4]
                    year = int(ddate[:4])
                    month = int(ddate[4:6])
                    day = int(ddate[6:])
                    ddate = datetime.date(year, month, day)
                    
                    quarter = which_quarter(ddate)
                    new_year = ddate.year

                    qtrs = line_list[5]
                    uom = line_list[6]
                    value = line_list[7]

                    more_terms = adsh_dict[adsh]
                    cik = more_terms[0] 
                    name = more_terms[1] 
                    filed = more_terms[2]  
                    
                    important_info = [adsh, tag, ddate, quarter, new_year, qtrs, uom, value, cik, name, filed]
                    
                    #if cik == "886158":
                    if cik == '40545':
                    #if True:
                        
                        try:
                            company_dict[cik]
                            
                            try:
                                company_dict[cik][1][new_year]
                                
                                try: 
                                    filed_date = company_dict[cik][1][new_year][quarter][2]
                                    
                                    if filed > filed_date:
                                        company_dict[cik][1][new_year][quarter] = [value, ddate, filed]       
                                
                                except:
                                    company_dict[cik][1][new_year][quarter] = [value, ddate, filed]
                            
                            except:
                                company_dict[cik][1][new_year] = {quarter: [value, ddate, filed]}
                                
                            
                        except:
                            company_dict[cik] = [name, {new_year: {quarter: [value, ddate, filed]}}]
                    
    
            except:
                error1 += 1
            counter += 1
    print("There were ",error1," lines that produced errors in ",file_to_open)
    return company_dict
    

In [135]:
def writing_to_file(company_dict: dict, write_file_name: str, tag: str) -> None: 
    for a_quarter in range(4):
        for a_year in range(2010,2020):
            for a_key in company_dict.keys():
                
                name = company_dict[a_key][0]
                
                csv_string = a_key + ", " + name + ", "
                
                try:
                    info_list = company_dict[a_key][1][a_year][a_quarter][:]
                    
                    value = float(info_list[0])
                    
                    ddate = info_list[1]
                    csv_string += str(ddate) + ", "
                    
                    csv_string += tag + ", "
                    
                    try:
                        old_values = []
                        for a_term in ['q','y','3y','5y']:
                            new_q, new_y = new_quarter(a_quarter, a_year, a_term)
                            try:
                                old_value = float(company_dict[a_key][1][new_y][new_q][0])
                            except:
                                old_value = None
                            old_values.append(old_value)
                        
                        counter = 0
                        for a_value in old_values:
                            if a_value == None or a_value == 0:
                                csv_string += ", "
                                counter += 1
                            else:
                                percentage = round((value - a_value) / a_value * 100 , 2)
                                csv_string += str(percentage) + ", "
                    except:
                        x = None
                    
                    if counter < 4:
                        csv_string += "\n"    
                        file = open(write_file_name ,'a')
                        file.writelines(csv_string)
                        file.close()
                                
                        
                        
                except:
                    x = None
                
    return

                    
                     
                
                

In [138]:
open_files = ['num10q1.txt', 'num10q2.txt', 'num10q3.txt', 'num10q4.txt', 'num11q1.txt', 'num11q2.txt', 'num11q3.txt', 'num11q4.txt', 'num12q1.txt', 'num12q2.txt', 'num12q3.txt', 'num12q4.txt', 'num13q1.txt', 'num13q2.txt', 'num13q3.txt', 'num13q4.txt', 'num14q1.txt', 'num14q2.txt', 'num14q3.txt', 'num14q4.txt', 'num15q1.txt', 'num15q2.txt', 'num15q3.txt', 'num15q4.txt', 'num16q1.txt', 'num16q2.txt', 'num16q3.txt', 'num16q4.txt', 'num17q1.txt', 'num17q2.txt', 'num17q3.txt', 'num17q4.txt', 'num18q1.txt', 'num18q2.txt', 'num18q3.txt', 'num18q4.txt', 'num19q1.txt', 'num19q2.txt']
    
sub_categories = ['Assets']
    
write_file_name = "percentage_growth.csv"

headers = "CIK, NAME, DDATE, MEASURE, QOQ_GROWTH, YOY_GROWTH, 3Y_GROWTH, 5Y_GROWTH\n"
    
file = open(write_file_name ,'a')
file.writelines(headers)
file.close()
    
t = time.time()

companies_dict = {}

num_files = 0
for a_category in sub_categories:    
    for a_file in open_files:
        try:
            companies_dict = cleaning_num_file(a_file, [a_category], num_files, companies_dict).copy()
        except:
            print(a_file," does not exist.")
            
        num_files += 1

    print("The dictionary was created in ",time.time()-t, " seconds.")
    
    t = time.time()
    
    writing_to_file(companies_dict, write_file_name, a_category)
    
    print("The file was written in ",time.time()-t, " seconds.")
    

There were  0  lines that produced errors in  num10q1.txt
There were  0  lines that produced errors in  num10q2.txt
There were  0  lines that produced errors in  num10q3.txt
There were  0  lines that produced errors in  num10q4.txt
There were  0  lines that produced errors in  num11q1.txt
There were  1  lines that produced errors in  num11q2.txt
There were  9  lines that produced errors in  num11q3.txt
There were  4  lines that produced errors in  num11q4.txt
There were  16  lines that produced errors in  num12q1.txt
There were  6  lines that produced errors in  num12q2.txt
There were  18  lines that produced errors in  num12q3.txt
There were  11  lines that produced errors in  num12q4.txt
There were  21  lines that produced errors in  num13q1.txt
There were  11  lines that produced errors in  num13q2.txt
There were  14  lines that produced errors in  num13q3.txt
There were  18  lines that produced errors in  num13q4.txt
There were  0  lines that produced errors in  num14q1.txt
There w